In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
from scipy.signal import butter, sosfiltfilt
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pathlib import Path
import lightgbm as lgb

from sklearn.metrics import f1_score

from PfyMU.gait.train_classifier.core import load_datasets
from PfyMU.features import *

plt.style.use('ggplot')

# Setup

In [3]:
def band_filter(x, fs):
    sos = butter(
        1, 
        [2 * 0.5 / fs, 2 * 3 / fs], 
        btype='band', 
        output='sos'
    )
    
    return sosfiltfilt(sos, x, axis=0)

steps = {
    'walking': 0.4,
    'walking-impaired': 0.2,
    'sitting': 900,
    'standing': 300,
    'stairs-ascending': 0.3,
    'stairs-descending': 0.3,
    'cycling-50W': 0.3,
    'cycling-100W': 0.3,
    'default': 1.0
}

# Load the Data

In [4]:
base_path = Path('/home/lukasadamowicz/Documents/Datasets/processed')

datasets = [
    base_path / 'bluesky2',
    base_path / 'daliac',
    base_path / 'ltmm',
    base_path / 'usc-had'
]

X_, Y, subjects, activities = load_datasets(
    paths=datasets,
    goal_fs=50.0,
    window_length=3.0,
    window_step=steps,
    acc_mag=False,
    signal_function=band_filter
)

In [5]:
x_nf, _, _, _ = load_datasets(
    paths=datasets,
    goal_fs=50.0,
    window_length=3.0,
    window_step=steps,
    acc_mag=False,
    signal_function=None
)

x_mn = Mean().compute(x_nf)

In [18]:
X = np.zeros(X_.shape[:-1])
for subj in np.unique(subjects):
    mask = (subjects == subj) & ((activities == 'walking-impaired') | (activities == 'walking'))
    if mask.sum() == 0:
        print('no mask')
    idx = np.argmax(np.abs(np.sum(x_mn[mask], axis=0)))
    
    if np.sum(x_mn[mask], axis=0)[idx] < 0:
        X[subjects == subj] = X_[subjects == subj, :, idx]
    else:
        X[subjects == subj] = -1 * X_[subjects == subj, :, idx]

# Randomize validation/test splits

In [14]:
random.seed(5)
rnd_subjects = [i for i in np.unique(subjects) if np.unique(activities[subjects==i]).size > 3]
random.shuffle(rnd_subjects)

training_masks, validation_masks, testing_masks = [], [], []

for i in range(0, len(rnd_subjects), 4):
    trm = np.ones(len(subjects), dtype='bool')
    vm = np.zeros_like(trm, dtype='bool')
    tem = np.zeros_like(trm, dtype='bool')
    
    for j in range(4):
        trm &= subjects != rnd_subjects[i + j]
        if j < 2:
            vm |= subjects == rnd_subjects[i + j]
        else:
            tem |= subjects == rnd_subjects[i + j]
    
    training_masks.append(trm)
    validation_masks.append(vm)
    testing_masks.append(tem)

# Feature Computation

In [15]:
FB = Bank(window_length=None, window_step=None)

FB + PowerSpectralSum(low_cutoff=0.0, high_cutoff=12.0)
FB + DominantFrequencyValue(low_cutoff=0.0, high_cutoff=12.0)
FB + MeanCrossRate()
FB + Range()
FB + RMS()
FB + SignalEntropy()
FB + SpectralEntropy(low_cutoff=0.0, high_cutoff=12.0)
FB + SpectralFlatness(low_cutoff=0.0, high_cutoff=12.0)

In [19]:
X_feat, fnames = FB.compute(X, fs=50.0, windowed=True, columns=[''])

# Model loading and testing

In [10]:
import pickle

with open('gaitpy_v1/model.pkl', 'rb') as f:
    model = pickle.load(f)

/home/lukasadamowicz/miniconda3/envs/pfymu/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/lukasadamowicz/miniconda3/envs/pfymu/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/lukasadamowicz/miniconda3/envs/pfymu/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Deci

# Cross validation

In [11]:
thresh = 0.5

In [24]:
fdf = pd.DataFrame(data=X_feat, columns=fnames)

In [39]:
f, ax = plt.subplots()
sns.boxplot(x='variable', y='value', data=fdf.melt(value_vars=fnames), ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
f1, tp, fp = [], [], []

print('Validation set performance')
for trm, vm, tem in zip(training_masks, validation_masks, testing_masks):
    y_pred = model.predict_proba(X_feat[vm])[:, 1]
    
    # compute metrics
    f1.append(f1_score(Y[vm], y_pred > thresh))
    tp.append((Y[vm] & (y_pred > thresh)).sum() / Y[vm].sum())
    fp.append((~Y[vm].astype(bool) & (y_pred > thresh)).sum() / (Y[vm].size - Y[vm].sum()))
    
    print(f'F1: {f1[-1]*100:6.1f}', end='')
    print(f'  TP: {tp[-1]*100:6.1f}', end='')
    print(f'  FP: {fp[-1]*100:6.1f}')
    
print('\n', '-' * 50)
print(f'Mean (SD) F1: {np.mean(f1)*100:.1f}({np.std(f1)*100:.1f})')
print(f'Mean (SD) TP: {np.mean(tp)*100:.1f}({np.std(tp)*100:.1f})')
print(f'Mean (SD) FP: {np.mean(fp)*100:.1f}({np.std(fp)*100:.1f})')

Validation set performance
F1:   12.5  TP:    7.1  FP:    5.0
F1:    0.0  TP:    0.0  FP:   18.7
F1:    3.8  TP:    2.5  FP:   13.6
F1:    0.0  TP:    0.0  FP:   25.1
F1:   27.4  TP:   16.1  FP:    3.1
F1:   14.8  TP:    8.0  FP:    0.9
F1:   49.4  TP:   33.5  FP:    1.5
F1:    1.5  TP:    0.8  FP:    0.7
F1:    0.6  TP:    0.4  FP:   22.0
F1:   36.0  TP:   23.1  FP:    2.5
F1:    0.0  TP:    0.0  FP:   23.1

 --------------------------------------------------
Mean (SD) F1: 13.3(16.3)
Mean (SD) TP: 8.3(10.8)
Mean (SD) FP: 10.6(9.5)


In [40]:
from gaitpy_v1 import signal_features

In [53]:
with open('gaitpy_v1/feature_order.txt', 'r') as f:
    lines = f.readlines()
lines = ['y_' + i.strip('\n').split('_', 5)[-1] for i in lines]


In [64]:
ix = 1

df = pd.DataFrame(data=X[ix].reshape((-1, 1)), columns=['y'])

res = signal_features._signal_features(df, ['y'], 50.0)
res = res[lines]
res.loc[1, :] = X_feat[ix]
res

,y_dom_freq_ratio,y_dom_freq_value,y_mean_cross_rate,y_range,y_rms,y_signal_entropy,y_spectral_entropy,y_spectral_flatness
0,0.295722,1.953125,0.126667,0.004001,0.000620,1.717011,0.762920,-6.993848
1,0.295889,0.121295,0.126667,0.004001,0.000622,1.688673,0.765209,-6.937629
